In [ ]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance


}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [ ]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
assistant_id = "asst_6g2V3a0J6kihX5oWsBnaNl2x"

# I want to know cloudflare stock is a good buy?

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

In [ ]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content= content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

# def get_tool_outputs(run_id, thread_id):
#     run = get_run(run_id, thread.id)
#     outputs = []
#     for action in run.required_action.submit_tool_outputs.tool_calls:
#         action_id = action.id
#         function = action.function
#         print(
#             f"Calling function: {function.name} with arg {function.arguments}")
#         outputs.append(
#             {
#                 "output": functions_map[function.name](json.loads(function.arguments)),
#                 "tool_call_id": action_id
#             }
#         )
#     return outputs

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(
            f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id
            }
        )
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id
        tool_outputs=outputs
    )

In [17]:
submit_tool_outputs(run.id, thread.id)

/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_11847/877756276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.retrieve(


Calling function: get_ticker with arg {"company_name": "Salesforce"}


/Users/junghaein/Documents/FULLSTACK_GPT/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


[{'output': "Aug 3, 2025 · Discover real-time Salesforce, Inc. Common Stock (CRM) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead … 1 day ago · The latest Salesforce stock prices, stock quotes, news, and CRM history to help you invest and trade smarter. Jan 8, 2025 · Find the latest historical data for Salesforce, Inc. Common Stock (CRM) at Nasdaq.com. View historical data in a monthly, bi-annual, or yearly format. Jun 13, 2025 · Get the latest Salesforce, Inc. (FRA:FOO) stock price with financials, statistics, dividends, charts and more. 5 days ago · Copado, the leader in AI-powered DevOps for business applications, today announced the launch of Org Intelligence, helping organizations discover, develop, and deploy Salesforce changes. Copado's ...",
  'tool_call_id': 'call_GQSWstbhXmgZqT5UxmLn4xgx'},
 {'output': '"{\\"1738281600000\\":{\\"Tax Effect Of Unusual Items\\":-98940000.0,\\"Tax Rate For Calcs\\":0.17,\\"Norma

In [18]:
get_run(run.id, thread.id).status

/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_11847/877756276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.retrieve(


'requires_action'

In [19]:
get_messages(thread.id)

/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_11847/877756276.py:15: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread_id)


user: I want to know if the Salesforce stock is a good buy
assistant: As an AI language model, I can provide you with information to help you conduct your own research on Salesforce (CRM) stock, but it's important to remember that investing in the stock market involves risk, and you should always do your due diligence before making any investment decisions.

To decide if Salesforce stock is a good buy, you might want to consider several key factors:

1. Financial Performance: Look at Salesforce's financial statements, including the income statement and balance sheet, to assess the company's profitability, revenue growth, expenses, and financial health.

2. Stock Performance: Examine the recent performance of Salesforce stock. Looking at the stock's historical price movements can give you insight into market sentiment and potential trends.

3. Fundamental Analysis: Evaluate fundamental indicators such as price-to-earnings (P/E) ratio, earnings per share (EPS), dividend yield (if any), a